In [35]:
from glob import glob
from PIL import Image
import numpy as np
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch import Tensor
from torchvision import datasets, transforms
from pytorch_fid import fid_score
from scipy.linalg import sqrtm
from scipy import linalg
from torchvision import transforms
from tqdm import tqdm 
from pytorch_fid.inception import InceptionV3
from torch.utils.data import DataLoader,Dataset
from torchvision.models import inception_v3
import os
print(f"GPUs used:\t{torch.cuda.device_count()}")
device = torch.device("cuda",4)
print(f"Device:\t\t{device}")

GPUs used:	8
Device:		cuda:4


In [36]:

params={'image_size':299,
        'data_path':'../../data/origin_type/STNT/유형1',
        'batch_size':1,
        }

trans = transforms.Compose([
            transforms.ToTensor(),

        ])

def transback(data:Tensor) -> Tensor:
    return data / 2 + 0.5

class CustomDataset(Dataset):
    """COCO Custom Dataset compatible with torch.utils.data.DataLoader."""
    def __init__(self,parmas, images):
        
        self.images = images
        self.args=parmas
        
    
    def __getitem__(self, index):
        image=self.images[index]
        return image
    
    def __len__(self):
        return len(self.images)
image_list=glob(params['data_path']+'/*.jpeg')

        
train_images=torch.zeros((len(image_list),3,params['image_size'],params['image_size']))
for i in tqdm(range(len(image_list))):
    train_images[i]=trans(Image.open(image_list[i]).convert('RGB').resize((params['image_size'],params['image_size'])))


100%|██████████| 735/735 [00:15<00:00, 47.68it/s]


In [37]:

# train_images=torch.rand((200,3,299,299)).to(device)
model = inception_v3(pretrained=True).to(device)
model.eval()
act=torch.zeros(len(train_images),1000)
for i in tqdm(range(len(train_images))):
    x=train_images[i].unsqueeze(0).to(device)
    act[i]=model(x)[0].detach().cpu()
act=act.numpy()

/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


100%|██████████| 735/735 [00:08<00:00, 88.18it/s]


In [41]:
mu1, sigma1=fid_score.calculate_activation_statistics(act[9:10])
mu2, sigma2=fid_score.calculate_activation_statistics(act[9:10])
fid=fid_score.calculate_frechet_distance(mu1, sigma1, mu2, sigma2)
fid

0.0

In [34]:
train_images.max()

tensor(2.6400)